In [ ]:
seed = 32
target_size = (380, 380)
batch_size = 16

In [ ]:
import pandas as pd
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
print(train.shape)
train['labels'].value_counts()

In [ ]:
#train_dir = "../input/resized-plant2021/img_sz_384"    #384*384
test_img = '../input/plant-pathology-2021-fgvc8/test_images'
submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submission.head()

將Train與Test分成 各12類別0.8/0.2

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
'''
label_list = train['labels'].value_counts().index
n_train = pd.DataFrame()
n_test = pd.DataFrame()
for i in label_list.tolist():
    i = train[train['labels']==i]
    x_train,x_test = train_test_split(i,test_size=0.2,random_state=CFG.seed)
    n_train = n_train.append(x_train)
    #n_train = sklearn.utils.shuffle(n_train)
    n_train.reset_index(drop=True, inplace=True)

    n_test = n_test.append(x_test)
    #n_test = sklearn.utils.shuffle(n_test)
    n_test.reset_index(drop=True, inplace=True)

print(n_train.shape)
print(n_test.shape)
print(n_train['labels'].head())
'''

Train轉為MultiLabel

In [ ]:
import numpy as np
import sklearn
from sklearn.preprocessing import MultiLabelBinarizer
mb = MultiLabelBinarizer()
labels = pd.DataFrame(mb.fit_transform(train.labels.apply(lambda x : x.split())), columns = mb.classes_)  
             
muti_train = pd.concat([train['image'], labels], axis=1)
print('muti_train')
print("-"*30)
for i in muti_train.columns[1:]:
    print(f"{i} : {np.count_nonzero(muti_train[i]==1)}")
print("-"*30)
muti_train

CV

In [ ]:
import cv2
import numpy as np
def get_cut_image(image):
    img = cv2.blur(image,(2,2))
    copy = np.uint8(img)
    canny = cv2.Canny(copy, 145, 165)
    box = np.argwhere(canny>0)
    y1,x1 = box.min(axis=0)
    y2,x2 = box.max(axis=0)
    cut_img = img[y1:y2, x1:x2]
    cut_img = cv2.resize(cut_img, target_size)
    #cut_img = cut_img.astype("float32")*(1.)/255
    return cut_img

### ImageDataGenerator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
'''
pre = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    brightness_range=[0.5, 1.5],
    rotation_range=45,
    shear_range=0.2,
    zoom_range=0.2, 
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2)
    #preprocessing_function = get_cut_image)
'''
pos =ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.5, 1.5],
    rotation_range=45,
    shear_range=0.2,
    zoom_range=0.2, 
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2)

In [ ]:
'''
train_generator = pre.flow_from_dataframe(
                  new_train,
                  directory = train_dir, 
                  x_col = 'image',
                  y_col = CFG.classes,
                  color_mode = "rgb",
                  target_size = (CFG.img_size,CFG.img_size),
                  class_mode = "raw",
                  batch_size = CFG.batch_size,
                  shuffle = True,
                  seed = CFG.seed
                  )

valid_generator = pre.flow_from_dataframe(
                  new_test,
                  directory = train_dir,
                  x_col = 'image',
                  y_col = CFG.classes,
                  color_mode = "rgb",
                  target_size = (CFG.img_size,CFG.img_size),
                  class_mode = "raw",
                  batch_size = CFG.batch_size,
                  shuffle = True,
                  seed = CFG.seed)
                  '''

In [ ]:
test_generator = pos.flow_from_dataframe(
                  submission,
                  directory = test_img,
                  x_col = 'image',
                  y_col = 'labels',
                  class_mode = "raw",
                  batch_size=batch_size,
                  target_size = target_size,
                  color_mode="rgb",
                  shuffle = False,
                  seed = seed)

In [ ]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
import keras
import tensorflow_addons as tfa
model = keras.models.load_model("../input/b4-4fold-ncv-bc1/B4_4fold_nCV_BC1.h5",compile=False)
pred = model.predict(test_generator)
pred

In [ ]:
perdict = (pred>0.33)
n_label = muti_train.columns.tolist()[1:]
answer = []

for i in range(perdict.shape[0]):
    temp = []
    for j, k in enumerate(n_label):
        if perdict[i, j]:
            temp.append(k)
    answer.append(temp)
    
answer = [' '.join(n) for n in answer]

In [ ]:
import numpy as np
submission['labels'] = np.array(answer)
submission

In [ ]:
submission.to_csv('./submission.csv', index=False)